In [ ]:
import pandas as pd
from common import load_stops
from merge_indicators import (
    calculate_accessibility_area_difference,
    determine_walk_area,
)

In [ ]:
CITY = "budapest"
VERSION = ""

In [ ]:
path = f"../output/{CITY}/{VERSION}"
distance = pd.read_csv(f"{path}/distance.csv", dtype={"stop_id": str})
distance.drop(["distance_from_largest_betweenness_medoid"], axis=1, inplace=True)
distance.rename(
    {"distance_from_largest_betweenness_centroid": "distance_betweenness"},
    axis=1,
    inplace=True,
)

ac = pd.read_csv(f"{path}/amenity_counts_in_accessibility.csv", dtype={"stop_id": str})
pt_ac = pd.read_csv(
    f"{path}/amenity_counts_in_public_transport_accessibility.csv",
    dtype={"stop_id": str},
)
stop_geometries = pd.read_csv(
    f"{path}/stop_geometries_from_walk.csv",
    dtype={"stop_id": str},
)

stop_centralities = load_stops(CITY, VERSION)
stop_centralities.drop(["Node", "geometry"], axis=1, inplace=True)
stop_centralities = stop_centralities.rename(
    {
        "Eigenvector Centrality": "eigenvector_centrality",
        "Degree Centrality": "degree_centrality",
        "Closeness Centrality": "closeness_centrality",
        "Betweenness Centrality": "betweenness_centrality",
        "clust": "cluster",
    },
    axis="columns",
)
stop_centralities.dropna(subset=["stop_id"], inplace=True)

In [95]:
wk_amenity = ac.query("costing == 'walk' & range == 15").copy()
wk_amenity.drop(["costing", "range"], axis=1, inplace=True)
wk_amenity.columns = ["stop_id"] + [f"{i}_walk15" for i in wk_amenity.columns[1:]]
mm_amenity = pt_ac.copy()
mm_amenity.drop(["costing", "range"], axis=1, inplace=True)
mm_amenity.columns = ["stop_id"] + [f"{i}_multimodal" for i in mm_amenity.columns[1:]]

In [ ]:
walk_area = determine_walk_area(CITY, VERSION)
walk_area = calculate_accessibility_area_difference(walk_area, stop_geometries)

In [ ]:
m = (
    stop_geometries.drop("geometry", axis=1)
    .merge(distance, on="stop_id")
    .merge(mm_amenity, on="stop_id")
    .merge(wk_amenity, on="stop_id")
    .merge(walk_area, on="stop_id")
    .merge(stop_centralities, on="stop_id")
)
m.to_csv(f"{path}/merged.csv", index=False)